### 项目描述
代码都写在py文件中，notebook仅作为一键运行的载体。

每个文件的主要功能为：
* solve*.py，对应6个问题的解法主程序
* ops.py，定义了若干矩阵运算算子
* utils.py，定义了基于numpy的模型构建函数，和基于0.618法的一维搜索函数等
* Reader.py，定义了矩阵文件和模型文件的读取函数
* paddle_model.py，定义了基于paddle的模型构建函数，基于反向传播的最优值求解函数等
* ArchitectureSearch.py，定义了随机搜索、抖动爬山法、模拟退火算法等架构搜索函数

### 第一题解题思路
第一题比较简单，做了个0.618法的一维搜索求最优值。

### 第二、三题解题思路
也比较简单，题目规定了电路结构，需要求解的是R电路的参数，参数量很小。首先通过若干轮随机搜索找到一个当前的最好解X，再在X的基础上逐步减少步长随机搜索以提高精度。

由于numpy不具备自动求导功能，无法得到有效的搜索方向，所以这里采用随机试探的方法。实际上也可以用后面题目的paddle模型自动求导求解，速度更快精度更高，只是代码懒得改了。

第二题耗时4s左右，第三题耗时8s左右。

### 第四、五、六题思路
从第四题开始，模型参数量变大，再采用随机搜索的方式求解参数变得不可取。采用paddle重新搭建电路模型，首先随机搜索电路结构，再自动求导解出当前结构对应的最优旋转角度。

经观察，同一量子连续经过几个旋转门可以等价为通过一个旋转门（其旋转角度等于几个旋转门的角度之和），所以旋转门连续排列是没有意义的。所以搜索时都是搜索一个旋转门（简称R门）跟着一个控制翻转门（简称C门），排列为RCRCRC...RCR的形式。

所以首先给定C门的个数n作为一个超参数，n限制了电路的长度和表达能力。C门的参数也随机指定。此时之需要通过paddle求解R门的参数即可。抖动爬山法（对应SequenceJitter函数）搜索时首先通过若干次随机搜索得到当前的最好解，再逐个尝试改变每一个C门的参数，以尝试得到更好的解。多轮抖动爬山后取全局最优解。

为了减少cost，采用了两个技巧。一是C门没有挡住的量子位（既不是控制位也不是翻转位）之后紧跟着的旋转门强制角度为0，打印电路时不再输出。因为它可以和前边的旋转门合并。二是对于搜索得到的最优解，尝试随机把旋转门置0，看能否在精度损失不大的情况下减少电路的消耗，提高总分数。

第五题提交的是采用模拟退火算法的答案，但采用抖动爬山搜索结果更优，耗时略长。如果延长模拟退火的轮次，应该也能得到更好的结果。

第四题耗时约7分钟，第五题耗时约20分钟。第六题按照目前的思路无法在可接受的时间内获得可行解（置信度高于0.75）。

### paddle模型的设计
相应代码见paddle_model.py，主要是RLayer_paddle、CLayer_paddle两个类。RLayer_paddle类自定义旋转角度为可优化的Parameter，forward时生成R门对应的响应矩阵，一个R门包括所有量子位的旋转门合并。CLayer_paddle类不包含参数。

再利用paddle.nn.Sequential整合为串行电路。模型的输入为单位阵I，输出为电路的0输入响应V，损失函数采用题目指定的置信度函数。

OptimizeModel函数为指定paddle模型，最优化模型参数（即R门的角度）。

BackwardParams函数根据指定的电路结构（即C门的设置），生成完整的paddle模型，再求解参数，并返回求得的电路。

### 准备数据
将数据解压到指定路径

In [4]:
### 在AI Studio上解压数据用，这里work目录下已有解压好的数据。
# !rm -rf work/Questions/*
# !unzip "data/data71784/飞桨常规赛：量子电路合成.zip" -d work/Questions
# !cd work/Questions/飞桨常规赛：量子电路合成/ && mv *.txt ../

### 运行主程序
在work目录下运行。

In [2]:
%cd ~/work/
!python solve1.py


In [7]:
!python solve2.py


In [9]:
!python solve3.py


In [10]:
!python solve4.py


In [8]:
!python solve5.py

In [5]:
# 随便跑一个结果
!python solve6.py


### 保存结果
将结果压缩包和相应的源代码保存在submits文件夹中，以分数命名。

In [9]:
!python check.py --save true

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 